In [1]:
import numpy as np
import progressbar

In [12]:
# honest network delay over next n blocks.
def vectorDelayHonest(ps, es, init_endorsers = 24, delay_priority = 40, delay_endorse = 8):
    return (60 * len(ps)
           + delay_priority * sum(ps) 
           + sum([delay_endorse * max(init_endorsers - e, 0) for e in es]))

# attacking network delay over next n blocks.
def vectorDelayAttacker(ps, es, init_endorsers = 24, delay_priority = 40, delay_endorse = 8):
    return (60 * len(ps) 
           + delay_priority * sum(ps) 
           + sum([delay_endorse * max(init_endorsers - e, 0) for e in es[1:]]))

# efficient sample generation
def getAH(alpha):
    x = np.random.geometric(1-alpha)
    if x == 1:
        h = 0
        a = np.random.geometric(alpha)
    else:
        a = 0
        h = x - 1
    return [a, h]

In [13]:
def getProbReorg(alpha, length, sample_size = int(1e5)):
    bar = progressbar.ProgressBar()
    feasible_count = 0
    for _ in bar(range(sample_size)):
        aVals = []
        hVals = []
        for i in range(length):
            a, h = getAH(alpha)
            aVals.append(a)
            hVals.append(h)
        eVals = np.random.binomial(32, alpha, size = length)
        honest_delay = vectorDelayHonest(hVals, 32 - eVals)
        selfish_delay = vectorDelayAttacker(aVals, eVals)
        if selfish_delay <= honest_delay:
            feasible_count += 1
    return feasible_count / sample_size

In [14]:
getProbReorg(alpha = 0.45, length = 20)

100% (100000 of 100000) |################| Elapsed Time: 0:00:09 Time:  0:00:09


0.05893